In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Sep  2 05:30:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#Upload the ExDark dataset


In [ ]:
!unzip /content/drive/MyDrive/yolo_project/datasets/ExDark.v1i.yolov8.zip -d /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/

Archive:  /content/drive/MyDrive/yolo_project/datasets/ExDark.v1i.yolov8.zip
replace /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Modify the data.yaml file
yaml_path = "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/data.yaml"

new_yaml_content = """
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 12
names:
  - Bicycle      # 0
  - Boat         # 1
  - People       # 2
  - Table        # 3
  - Bottle       # 4
  - Bus          # 5
  - Car          # 6
  - Cat          # 7
  - Chair        # 8
  - Cup          # 9
  - Dog          # 10
  - Motorbike    # 11
"""

with open(yaml_path, "w") as f:
    f.write(new_yaml_content)


# Image enhancement

## CLAHE and Gamma Lightweight Image Enhancement

In [ ]:
import cv2
import os
import numpy as np
from glob import glob
from tqdm import tqdm

# Image enhancement function
def enhance_image(img, gamma=1.5, clahe_clip=2.0, clahe_grid=(8, 8)):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clahe_clip, tileGridSize=clahe_grid)
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    img_clahe = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in range(256)], dtype=np.uint8)
    img_gamma = cv2.LUT(img_clahe, table)
    return img_gamma

# List of dataset paths
splits = ["train", "valid", "test"]
base_input = "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO"
base_output = "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_enhanced"

# Split processing one by one
for split in splits:
    input_folder = os.path.join(base_input, split, "images")
    output_folder = os.path.join(base_output, split, "images")
    os.makedirs(output_folder, exist_ok=True)

    image_paths = glob(os.path.join(input_folder, "*.jpg"))
    print(f"Enhancing {split} images: {len(image_paths)} found")

    for img_path in tqdm(image_paths, desc=f"Enhancing {split}"):
        img = cv2.imread(img_path)
        if img is not None:
            enhanced = enhance_image(img)
            filename = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_folder, filename), enhanced)

🟢 Enhancing train images: 5141 found


Enhancing train: 100%|██████████| 5141/5141 [05:47<00:00, 14.77it/s]


🟢 Enhancing valid images: 1469 found


Enhancing valid: 100%|██████████| 1469/1469 [01:40<00:00, 14.56it/s]


🟢 Enhancing test images: 734 found


Enhancing test: 100%|██████████| 734/734 [00:41<00:00, 17.49it/s]


In [ ]:
!cp -r /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/train/labels /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_enhanced/train/
!cp -r /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/valid/labels /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_enhanced/valid/
!cp -r /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/test/labels /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_enhanced/test/

##ExDark_CLAHE lightweight image enhancement

In [ ]:
import os
import cv2
from tqdm import tqdm

input_dirs = [
    "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/train/images",
    "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/valid/images",
    "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/test/images"
]

output_dirs = [
    "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/train/images",
    "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/valid/images",
    "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/test/images"
]

for out_dir in output_dirs:
    os.makedirs(out_dir, exist_ok=True)

# CLAHE image enhancement function
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

# Perform augmentation on each dataset subset (train/valid/test)
for input_dir, output_dir in zip(input_dirs, output_dirs):
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"Processing {input_dir} -> {output_dir} | Total: {len(image_files)} images")

    for image_name in tqdm(image_files):
        img_path = os.path.join(input_dir, image_name)
        out_path = os.path.join(output_dir, image_name)

        image = cv2.imread(img_path)
        if image is None:
            continue  # Skip corrupted images
        enhanced = apply_clahe(image)
        cv2.imwrite(out_path, enhanced)

print("All CLAHE enhancement completed.")

Processing /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/train/images -> /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/train/images | Total: 5141 images


100%|██████████| 5141/5141 [05:46<00:00, 14.82it/s]


Processing /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/valid/images -> /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/valid/images | Total: 1469 images


100%|██████████| 1469/1469 [01:55<00:00, 12.76it/s]


Processing /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/test/images -> /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/test/images | Total: 734 images


100%|██████████| 734/734 [00:43<00:00, 16.89it/s]

All CLAHE enhancement completed.


In [ ]:
import shutil
import os

original_root = "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO"

target_root = "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE"

splits = ['train', 'valid', 'test']

for split in splits:
    src = os.path.join(original_root, split, "labels")
    dst = os.path.join(target_root, split, "labels")
    os.makedirs(dst, exist_ok=True)
    for file in os.listdir(src):
        shutil.copy(os.path.join(src, file), os.path.join(dst, file))

print("Label copying completed!")


Label copying completed!


In [ ]:
data_yaml_path = "/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/data.yaml"

yaml_content = """\
train: /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/train/images
val: /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/valid/images
test: /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/test/images

nc: 12
names: ['Bicycle', 'Boat', 'People', 'Table', 'Bottle', 'Bus', 'Car', 'Cat', 'Chair', 'Cup', 'Dog', 'Motorbike']
"""

with open(data_yaml_path, 'w') as f:
    f.write(yaml_content)

print("The new data.yaml file has been created!")

The new data.yaml file has been created!


## ExDark_gamma lightweight image enhancement

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Gamma enhancement function
def gamma_correction(img, gamma=1.5):
    invGamma = 1.0 / gamma
    table = np.array([
        ((i / 255.0) ** invGamma) * 255 for i in np.arange(256)
    ]).astype("uint8")
    return cv2.LUT(img, table)

input_root = '/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO'
output_root = '/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_Gamma'
splits = ['train', 'valid', 'test']

for split in splits:
    input_dir = os.path.join(input_root, split, 'images')
    output_dir = os.path.join(output_root, split, 'images')
    os.makedirs(output_dir, exist_ok=True)

    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Processing {split} set: {len(image_files)} images...")

    for file in tqdm(image_files):
        img_path = os.path.join(input_dir, file)
        out_path = os.path.join(output_dir, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        enhanced = gamma_correction(img, gamma=1.5)
        cv2.imwrite(out_path, enhanced)

print("Gamma image enhancement completed, results saved to ExDark_YOLO_Gamma.")

Processing train set: 5141 images...


100%|██████████| 5141/5141 [05:37<00:00, 15.26it/s]


Processing valid set: 1469 images...


100%|██████████| 1469/1469 [02:11<00:00, 11.17it/s]


Processing test set: 734 images...


100%|██████████| 734/734 [00:31<00:00, 23.07it/s]

Gamma image enhancement completed, results saved to ExDark_YOLO_Gamma.


In [ ]:
import shutil
import os

original_labels_root = '/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO'

retinex_labels_root = '/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_Gamma'

# Copy train, valid, and test separately
splits = ['train', 'valid', 'test']
for split in splits:
    src = os.path.join(original_labels_root, split, 'labels')
    dst = os.path.join(retinex_labels_root, split, 'labels')
    os.makedirs(dst, exist_ok=True)

    for file in os.listdir(src):
        shutil.copy(os.path.join(src, file), os.path.join(dst, file))

print("Label copying completed: train, valid, and test labels have been synchronized to the Gamma dataset.")

Label copying completed: train, valid, and test labels have been synchronized to the Gamma dataset.


# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.192 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 39.0/225.8 GB disk)


#Start training dataset

##Dataset before training without augmentation

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer='SGD',
    close_mosaic=10,
    device='0',
    workers=0,
    cache=False,
    project='runs/train',
    name='exdark_experiment 1.0'
)

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exdark_experiment 1.0, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=100

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c5509bd6e10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [ ]:
# Save the run results to My Drive
!cp -r /content/runs/train/exdark_experiment 1.0 /content/drive/MyDrive/yolo_project/outputs/

cp: cannot stat '/content/runs/train/exdark_experiment': No such file or directory
cp: cannot stat '1.0': No such file or directory


In [ ]:
# Check if the dataset is complete
!ls /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/train/images | wc -l
!ls /content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO/valid/images | wc -l

5141
1469


## Training image augmented dataset

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # use pretrained base model

model.train(
    data='/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_enhanced/exdark_enhanced.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer='SGD',
    close_mosaic=10,
    device='0',
    workers=0,
    cache=False,
    project='runs/train',
    name='exdark_enhanced_experiment 1.0'
)

Ultralytics 8.3.187 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_enhanced/exdark_enhanced.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exdark_enhanced_experiment 1.0, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, ov

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c55a88d4920>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [ ]:
!cp -r /content/runs/train/exdark_enhanced_experiment /content/drive/MyDrive/yolo_project/outputs

In [4]:
from ultralytics import YOLO

# Loading the trained model
model = YOLO("/content/drive/MyDrive/yolo_project/outputs/experiments/exdark_experiment/weights/best.pt")

# Predict an image and save the result
results = model.predict(source="/content/drive/MyDrive/yolo_project/test_images/test1.jpg", save=True)


image 1/1 /content/drive/MyDrive/yolo_project/test_images/test1.jpg: 640x480 1 People, 1 Bus, 5 Cars, 266.8ms
Speed: 25.2ms preprocess, 266.8ms inference, 37.8ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict


In [5]:
from ultralytics import YOLO

# Loading the trained model
model = YOLO("/content/drive/MyDrive/yolo_project/outputs/experiments/exdark_gamma_experimen/weights/best.pt")

# Predict an image and save the result
results = model.predict(source="/content/drive/MyDrive/yolo_project/test_images/test1.jpg", save=True)


image 1/1 /content/drive/MyDrive/yolo_project/test_images/test1.jpg: 640x480 1 People, 5 Cars, 54.2ms
Speed: 2.0ms preprocess, 54.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict2


In [ ]:
from ultralytics import YOLO

# Loading a pre-trained model
model = YOLO('yolov8n.pt')

# Training enhanced ExDark dataset (CLAHE enhanced version)
model.train(
    data='/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer='SGD',
    close_mosaic=10,
    device='0',
    workers=0,
    cache=False,
    project='runs/train',
    name='exdark_experiment_CLAHE'
)

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_CLAHE/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exdark_experiment_CLAHE, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patie

Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,013,188 parameters, 3,013,172 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 1.0±0.2 ms, read: 0.2±0.1 MB/s, size: 98.1 KB)


train: Scanning /content/drive/.shortcut-targets-by-id/1BYfwhwwplTKb5Jtj4o4UX6FpLNWNerHa/yolo_project/datasets/ExDark_YOLO_CLAHE/train/labels... 5141 images, 1 backgrounds, 0 corrupt: 100%|██████████| 5141/5141 [45:19<00:00,  1.89it/s]


...

                   all       1469       4659       0.77      0.617      0.699      0.439



100 epochs completed in 5.186 hours.
Optimizer stripped from runs/train/exdark_experiment_CLAHE/weights/last.pt, 6.2MB
Optimizer stripped from runs/train/exdark_experiment_CLAHE/weights/best.pt, 6.2MB

Validating runs/train/exdark_experiment_CLAHE/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:24<00:00,  1.92it/s]


                   all       1469       4659       0.76      0.616      0.706      0.446
               Bicycle        146        231       0.83      0.635      0.746      0.469
                  Boat        155        293      0.742      0.647      0.745       0.38
                People        529       1502        0.8      0.602      0.718      0.413
                 Table        187        290      0.624      0.503      0.549       0.33
                Bottle        136        293      0.776      0.603      0.679      0.428
                   Bus         99        119       0.81      0.849      0.918      0.693
                   Car        261        606       0.82      0.667      0.762       0.49
                   Cat        151        173      0.754      0.647      0.726      0.457
                 Chair        234        435      0.731      0.485      0.589      0.373
                   Cup        162        303      0.678      0.498      0.596      0.393
                   Do

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d5bf53d9d90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [ ]:
!cp -r /content/runs/train/exdark_experiment_CLAHE /content/drive/MyDrive/yolo_project/datasets

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_Retinex/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer='SGD',
    close_mosaic=10,
    device='0',
    workers=0,
    cache=False,
    project='runs/train',
    name='exdark_retinex_experiment'
)

100%|██████████| 6.25M/6.25M [00:00<00:00, 367MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_Retinex/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exdark_retinex_experiment, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap

100%|██████████| 755k/755k [00:00<00:00, 140MB/s]

Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,013,188 parameters, 3,013,172 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

                   all       1469       4659       0.73      0.615      0.686      0.427



100 epochs completed in 7.366 hours.
Optimizer stripped from runs/train/exdark_retinex_experiment/weights/last.pt, 6.2MB
Optimizer stripped from runs/train/exdark_retinex_experiment/weights/best.pt, 6.2MB

Validating runs/train/exdark_retinex_experiment/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [00:19<00:00,  2.37it/s]


                   all       1469       4659      0.721      0.626      0.689      0.431
               Bicycle        146        231      0.769      0.641      0.728      0.455
                  Boat        155        293      0.712      0.638      0.726      0.384
                People        529       1502      0.757      0.625      0.713      0.404
                 Table        187        290      0.638       0.51      0.551      0.316
                Bottle        136        293      0.792      0.583      0.649      0.414
                   Bus         99        119      0.823      0.891      0.916      0.677
                   Car        261        606      0.784      0.678      0.759      0.483
                   Cat        151        173      0.647      0.613      0.657      0.409
                 Chair        234        435      0.681      0.503      0.589      0.363
                   Cup        162        303      0.622      0.558      0.604      0.394
                   Do

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b2e399afd10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [ ]:
!cp -r /content/runs/train/exdark_retinex_experiment /content/drive/MyDrive/yolo_project/outputs

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.train(
    data='/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_Gamma/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    optimizer='SGD',
    close_mosaic=10,
    device='0',
    workers=0,
    project='runs/train',
    name='exdark_gamma_experimen'
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/datasets/ExDark_YOLO_Gamma/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exdark_gamma_experimen, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patie

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fdd95a03560>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,  

In [ ]:
# 将运行结果保存到My Drive中
!cp -r /content/runs/train/exdark_gamma_experiment /content/drive/MyDrive/yolo_project/outputs

# Upload the BDD dataset

In [ ]:
!unzip -q /content/drive/MyDrive/yolo_project/datasets/bdd100k_images.zip -d /content/drive/MyDrive/yolo_project/datasets/BDD100K
!unzip -q /content/drive/MyDrive/yolo_project/datasets/bdd100k_labels.zip -d /content/drive/MyDrive/yolo_project/datasets/BDD100K

In [ ]:
!ls /content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/images/100k/train | wc -l
!ls /content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/images/100k/val | wc -l

70000
10000


##BDD100K_Night

In [ ]:
import os
import json
import shutil
from tqdm import tqdm

# Setting the path
json_dir = "/content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/labels/100k/train"
image_dir = "/content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/images/100k/train"
target_image_dir = "/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/images/train"
target_label_dir = "/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/train"

# Create a destination folder
os.makedirs(target_image_dir, exist_ok=True)
os.makedirs(target_label_dir, exist_ok=True)

count = 0

# Iterate over all JSON files
for json_file in tqdm(os.listdir(json_dir)):
    if not json_file.endswith(".json"):
        continue
    json_path = os.path.join(json_dir, json_file)
    with open(json_path, "r") as f:
        data = json.load(f)

    # Check if it is night scene
    if data.get("attributes", {}).get("timeofday") == "night":
        img_name = json_file.replace(".json", ".jpg")
        img_src = os.path.join(image_dir, img_name)
        label_src = json_path

        img_dst = os.path.join(target_image_dir, img_name)
        label_dst = os.path.join(target_label_dir, json_file)

        # Copy images and labels
        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)
            shutil.copy(label_src, label_dst)
            count += 1

print(f"Extraction completed! Total night images extracted: {count}")


100%|██████████| 59894/59894 [6:43:58<00:00,  2.47it/s]

Extraction completed! Total night images extracted: 23922


In [ ]:
import os
import json
import shutil
from tqdm import tqdm

val_images_path = '/content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/images/100k/val'
val_labels_path = '/content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/labels/100k/val'

target_val_images = '/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/images/val'
target_val_labels = '/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/val'

os.makedirs(target_val_images, exist_ok=True)
os.makedirs(target_val_labels, exist_ok=True)

night_count = 0
for filename in tqdm(os.listdir(val_labels_path)):
    if filename.endswith('.json'):
        json_path = os.path.join(val_labels_path, filename)
        with open(json_path, 'r') as f:
            data = json.load(f)
            if data.get('attributes', {}).get('timeofday') == 'night':
                base_name = filename.replace('.json', '.jpg')

                src_img_path = os.path.join(val_images_path, base_name)
                dst_img_path = os.path.join(target_val_images, base_name)

                src_lbl_path = json_path
                dst_lbl_path = os.path.join(target_val_labels, filename)

                if os.path.exists(src_img_path):
                    shutil.copy2(src_img_path, dst_img_path)
                    shutil.copy2(src_lbl_path, dst_lbl_path)
                    night_count += 1

print(f"Extraction completed! Total night images in val set: {night_count}")

100%|██████████| 10000/10000 [05:13<00:00, 31.95it/s]

Extraction completed! Total night images in val set: 3929


In [ ]:
import os

label_path = "/content/drive/MyDrive/yolo_project/datasets/BDD100K/bdd100k/labels/100k/train"

# Get all json files in the directory
json_files = [f for f in os.listdir(label_path) if f.endswith('.json')]

print(f"Found {len(json_files)} JSON files")
if json_files:
    print("First 5 file names：", json_files[:5])
else:
    print("⚠️ JSON file not found, please check if the path is correct or if the file has been loaded。")


Found 70000 JSON files
First 5 file names： ['0000f77c-6257be58.json', '0000f77c-62c2a288.json', '0000f77c-cb820c98.json', '0001542f-5ce3cf52.json', '0001542f-7c670be8.json']


In [ ]:
import os

json_dir = '/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/train'
print(os.listdir(json_dir))

['067f3361-39097fb1.json', '05584a52-730a88df.json', '05b3562f-6a5ba981.json', '058a26dd-131118a0.json', '06727591-f55ebbfa.json', '0568273b-0fb92589.json', '05f29ddd-eda326ca.json', '06770a18-a93978ab.json', '0568da9f-a9e68991.json', '06668f7b-81a4f8e5.json', '05e61937-4d3e06ad.json', '05d30a18-19032166.json', '066a3f28-bedf609f.json', '0554d0e1-545facff.json', '0678f15f-a4ca2b42.json', '064bad4e-e26c64f8.json', '064bad4e-ae51d829.json', '05ed3eee-bde322cc.json', '06844897-e035260a.json', '068ac51e-c65ca775.json', '05b3562f-921788d0.json', '06265565-67924b70.json', '0679098c-8952ba69.json', '0598d1c5-1445b586.json', '06147f21-9357dcfc.json', '057e248a-d682ca97.json', '05af6dc5-5baee748.json', '066cf59c-d989b300.json', '05fd5ca5-d5aa90ab.json', '0598d1c5-932bc43d.json', '05d1493d-56495b31.json', '05f4413f-bae0626e.json', '0554d0e1-cc67ac36.json', '06924f46-7f03bcdc.json', '05c5f168-4102d5dc.json', '05d481cb-cdf61aa2.json', '05c1589a-60d8d1f3.json', '0554d0e1-2a24a6bd.json', '06583aaf-b

In [ ]:
import os

# Check the number of files in each folder
train_images = os.listdir('/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/images/train')
train_labels = os.listdir('/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/train')

val_images = os.listdir('/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/images/val')
val_labels = os.listdir('/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/val')

print(f'Found {len(train_images)} train images and {len(train_labels)} train labels.')
print(f'Found {len(val_images)} val images and {len(val_labels)} val labels.')

Found 23922 train images and 23922 train labels.
Found 3929 val images and 3929 val labels.


In [ ]:
import os
import json
from collections import defaultdict

def analyze_bdd100k_categories(json_dir):
    category_count = defaultdict(int)

    for filename in os.listdir(json_dir):
        if not filename.endswith('.json'):
            continue
        path = os.path.join(json_dir, filename)
        with open(path, 'r') as f:
            data = json.load(f)
            if 'frames' in data and len(data['frames']) > 0:
                for obj in data['frames'][0].get('objects', []):
                    if 'category' in obj:
                        category_count[obj['category']] += 1

    print("Found categories:")
    for k, v in sorted(category_count.items(), key=lambda x: -x[1]):
        print(f"{k}: {v} instances")

analyze_bdd100k_categories('/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/train')

Found categories:
car: 218126 instances
lane/single white: 79873 instances
traffic sign: 76335 instances
traffic light: 71873 instances
lane/crosswalk: 35832 instances
lane/road curb: 32303 instances
area/drivable: 22305 instances
area/alternative: 20451 instances
person: 16008 instances
lane/double yellow: 11756 instances
truck: 4559 instances
lane/single yellow: 3905 instances
lane/double white: 2245 instances
bus: 1884 instances
bike: 1286 instances
rider: 742 instances
motor: 555 instances
train: 27 instances
lane/single other: 23 instances
area/unknown: 1 instances
lane/double other: 1 instances


## Configuring Category Mapping

In [ ]:
import os
import json
from tqdm import tqdm

# Only these 5 categories are retained and mapped to the ID in YOLO
CATEGORY_MAP = {
    "car": 6,      # 0 - corresponds to Car in ExDark
    "bus": 5,      # 1 - corresponds to Bus in ExDark
    "person": 2,   # 2 - corresponds to Person in ExDark
    "bike": 0,     # 3 - Corresponding to Bicycle in ExDark
    "motor": 11    # 4 - Corresponds to Motorbike in ExDark
}

# Image size (based on BDD100K default resolution, 1280x720)
IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 720

## Convert JSON tags to YOLO format

In [ ]:
def convert_bdd100k_to_yolo(json_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

    for file in tqdm(json_files, desc=f"Converting {json_dir}"):
        with open(os.path.join(json_dir, file), 'r') as f:
            data = json.load(f)

        yolo_labels = []

        for frame in data.get('frames', []):
            for obj in frame.get('objects', []):
                cat = obj.get('category')
                if cat not in CATEGORY_MAP:
                    continue

                if 'box2d' not in obj:
                    continue

                box = obj['box2d']
                x1, y1 = box['x1'], box['y1']
                x2, y2 = box['x2'], box['y2']

                # Convert to YOLO format
                x_center = (x1 + x2) / 2.0 / IMAGE_WIDTH
                y_center = (y1 + y2) / 2.0 / IMAGE_HEIGHT
                width = (x2 - x1) / IMAGE_WIDTH
                height = (y2 - y1) / IMAGE_HEIGHT

                class_id = CATEGORY_MAP[cat]
                yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        out_path = os.path.join(output_dir, file.replace('.json', '.txt'))
        with open(out_path, 'w') as out_f:
            out_f.write('\n'.join(yolo_labels))

In [ ]:
base_path = '/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels'
output_base = '/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels_yolo'

convert_bdd100k_to_yolo(os.path.join(base_path, 'train'), os.path.join(output_base, 'train'))
convert_bdd100k_to_yolo(os.path.join(base_path, 'val'), os.path.join(output_base, 'val'))

print("JSON tags have been converted to YOLO format!")

Converting /content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/train: 100%|██████████| 23922/23922 [18:42<00:00, 21.31it/s]
Converting /content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels/val: 100%|██████████| 3929/3929 [04:15<00:00, 15.37it/s]

JSON tags have been converted to YOLO format!


In [ ]:
import os
val_labels_path = '/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/labels_yolo/val'
print(os.listdir(val_labels_path))

['c4aac9d9-47b01027.txt', 'c4ccffb1-e4215665.txt', 'c4dab4e2-6c4805a2.txt', 'c4d7185b-6c141918.txt', 'c4d023f4-27b83fe1.txt', 'c4cb6f64-283904da.txt', 'c4db746c-5428658b.txt', 'c4a9c6f0-f18e20e1.txt', 'c4ccd765-48590402.txt', 'c4d25f00-b6658057.txt', 'c4d25f00-158d0757.txt', 'c4d1be5c-f868e120.txt', 'c4c5f66a-a1df8079.txt', 'c4b610ae-4b915923.txt', 'c4dbd719-1d98586e.txt', 'c4af4030-d3b1831d.txt', 'c4b610ae-bc6dc0c9.txt', 'c4c35028-acf2331c.txt', 'c4cc2bbd-a4913bdf.txt', 'c4ccffb1-e53dc8ca.txt', 'c4e5de4b-35f6a1b1.txt', 'c4ee0cc4-b7af9602.txt', 'c4fb5145-f7056d01.txt', 'c4f993bf-e6edd387.txt', 'c4e2eba0-f32cc5b6.txt', 'c4f723f1-a0aab25c.txt', 'c4fa3b0a-29b96446.txt', 'c4e8705f-39bf4a26.txt', 'c4f723f1-babfc6fe.txt', 'c4e00603-9f2734d0.txt', 'c4f723f1-5ba8f7ee.txt', 'c4f993bf-3eee07b1.txt', 'c4e92086-e0ea5438.txt', 'c50193e1-630410dc.txt', 'c517c182-d75f08e3.txt', 'c524ecb5-74c5e510.txt', 'c51d0636-89b6a9d9.txt', 'c51a2815-5d5e6d1a.txt', 'c51d0636-2b91b311.txt', 'c50c20fc-7941d80e.txt',

# Find the best epoch indicator for each training result

In [ ]:
import os, json
import pandas as pd

root = "/content/drive/MyDrive/yolo_project/outputs/experiments"  # The parent directory, each subfolder in it is an experiment
rows = []

for sub in sorted(os.listdir(root)):
    exp_dir = os.path.join(root, sub)
    csv_path = os.path.join(exp_dir, "results.csv")
    if not os.path.isfile(csv_path):
        continue
    df = pd.read_csv(csv_path)

    candidates = [
        ("metrics/precision(B)", "metrics/recall(B)", "metrics/mAP50(B)", "metrics/mAP50-95(B)"),
        ("metrics/precision",    "metrics/recall",    "metrics/mAP50",    "metrics/mAP50-95"),
    ]
    ok = False
    for cols in candidates:
        if all(c in df.columns for c in cols):
            P_col, R_col, m50_col, m5095_col = cols
            ok = True
            break
    if not ok:
        print(f"Skip (column name mismatch):{csv_path}")
        continue

    best = df.loc[df[m5095_col].idxmax()]
    rows.append({
        "experiment": sub,
        "epoch": int(best["epoch"]) if "epoch" in df.columns else None,
        "precision": float(best[P_col]),
        "recall": float(best[R_col]),
        "mAP@0.5": float(best[m50_col]),
        "mAP@0.5:0.95": float(best[m5095_col]),
    })

summary = pd.DataFrame(rows).sort_values("mAP@0.5:0.95", ascending=False)
out_csv = os.path.join(root, "summary_best.csv")
summary.to_csv(out_csv, index=False)
summary
print(f"Summary saved:{out_csv}")


Summary saved:/content/drive/MyDrive/yolo_project/outputs/experiments/summary_best.csv


# Fine-tuning BDD100K-Night


In [ ]:
!rsync -a --info=progress2 "/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night" "/content/BDD100K_Night/"

     33,647,356   2%   50.97kB/s    0:10:44 (xfr#843, ir-chk=27009/27858)
rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at rsync.c(716) [sender=3.2.7]
rsync: [generator] write error: Broken pipe (32)
rsync error: received SIGINT, SIGTERM, or SIGHUP (code 20) at io.c(519) [receiver=3.2.7]
rsync: connection unexpectedly closed (809469 bytes received so far) [generator]


In [ ]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/yolo_project/outputs/exdark_experiment 2.0/weights/best.pt')

model.train(
    data='/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/data.yaml',
    epochs=50,
    imgsz=640,
    batch=8,
    optimizer='SGD',
    device='0',
    workers=2,
    cache=False,
    project='runs/train',
    name='bdd100k_night_finetune'
)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_project/datasets/BDD100K_Night/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/yolo_project/outputs/exdark_experiment 2.0/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bdd100k_night_finetune2, nbs=64, nms=False, op

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  2,  5,  6, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x787234dab830>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
   

# COCO-Tiny dataset, for generality testing

In [ ]:
from ultralytics import YOLO
import os, glob, shutil

if not os.path.exists('/content/datasets/coco128'):
    YOLO().val(data='coco128.yaml', imgsz=64, batch=1, device='cpu')

SRC_IMG_DIR = '/content/datasets/coco128/images/train2017'
SRC_LAB_DIR = '/content/datasets/coco128/labels/train2017'

DST_ROOT    = '/content/coco128_exdark5'
DST_IMG_DIR = f'{DST_ROOT}/images/val'
DST_LAB_DIR = f'{DST_ROOT}/labels/val'

shutil.rmtree(DST_ROOT, ignore_errors=True)
os.makedirs(DST_IMG_DIR, exist_ok=True)
os.makedirs(DST_LAB_DIR, exist_ok=True)

# Only keep and remap these 5 categories: person, bicycle, car, motorcycle, bus
# Map to ExDark's 12 category indexes: People(2), Bicycle(0), Car(6), Motorbike(11), Bus(5)
MAP = {0:2, 1:0, 2:6, 3:11, 5:5}

def remap_one(src_txt, dst_txt):
    lines_out = []
    if not os.path.exists(src_txt):
        open(dst_txt, 'w').close()
        return 0

    with open(src_txt, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 5:
                continue
            cid = int(parts[0])
            if cid in MAP:
                parts[0] = str(MAP[cid])
                lines_out.append(' '.join(parts))

    with open(dst_txt, 'w') as f:
        f.write('\n'.join(lines_out))
    return len(lines_out)

# Copy image + rewrite label
from tqdm import tqdm
kept, total = 0, 0
for img_path in tqdm(glob.glob(os.path.join(SRC_IMG_DIR, '*.jpg')), desc='Remap COCO128->ExDark5'):
    fname = os.path.basename(img_path)
    stem  = os.path.splitext(fname)[0]
    lab_src = os.path.join(SRC_LAB_DIR, stem + '.txt')
    lab_dst = os.path.join(DST_LAB_DIR,  stem + '.txt')

    shutil.copy2(img_path, os.path.join(DST_IMG_DIR, fname))
    n = remap_one(lab_src, lab_dst)
    kept += (n > 0)
    total += 1

print(f'Remap done: {total} images, images with labels (≥1 box): {kept}')

# Simple sampling of several label files
import random
samples = random.sample(glob.glob(os.path.join(DST_LAB_DIR, '*.txt')), 5)
for p in samples:
    with open(p, 'r') as f:
        first = f.readline().strip()
    print(os.path.basename(p), '->', first)

# Write data.yaml (ExDark 12-class order)
exdark12 = [
 'Bicycle','Boat','People','Table','Bottle','Bus','Car','Cat','Chair','Cup','Dog','Motorbike'
]
yaml_text = f"""# coco128 remapped to ExDark(12) subset
train: {DST_IMG_DIR}
val: {DST_IMG_DIR}

nc: 12
names: {exdark12}
"""
with open(f'{DST_ROOT}/data.yaml','w') as f:
    f.write(yaml_text)

# Evaluate
model = YOLO('/content/drive/MyDrive/yolo_project/outputs/bdd100k_night_finetune2/weights/best.pt')  # 改成你的权重
model.val(data=f'{DST_ROOT}/data.yaml', imgsz=640, batch=16, device=0)

Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

WARNING ⚠️ Dataset 'coco128.yaml' images not found, missing path '/content/datasets/coco128/images/train2017'
Unzipping /content/datasets/coco128.zip to /content/datasets/coco128...: 100% ━━━━━━━━━━━━ 263/263 4497.6files/s 0.1s
Dataset download success ✅ (1.2s), saved to /content/datasets

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1783.9±755.7 MB/s, size: 56.3 KB)
val: Scanning /content/datasets/coco128/labels/train2017... 126 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 1096.1it/s 0.1s
val: New cache created: /content/datasets/coco128/labels/train2017.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 128/128 68.8it/s 1.9s
                   all        128        929     0.0351      0.126     0.0794     0.0459
                person       

Remap COCO128->ExDark5: 100%|██████████| 128/128 [00:00<00:00, 4876.92it/s]

✅ Remap done: 128 images, 有标签(≥1 框)的图片: 71
000000000428.txt -> 2 0.632656 0.539681 0.556125 0.897972
000000000650.txt -> 6 0.501859 0.820726 0.996281 0.332178
000000000569.txt -> 2 0.200297 0.712146 0.125281 0.302875
000000000560.txt -> 
000000000338.txt -> 2 0.581609 0.715352 0.141344 0.561774


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1836.2±766.6 MB/s, size: 62.7 KB)
val: Scanning /content/coco128_exdark5/labels/val... 128 images, 57 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 1887.4it/s 0.1s
val: New cache created: /content/coco128_exdark5/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 3.4it/s 2.4s
                   all        128        318      0.544      0.166      0.222      0.135
               Bicycle          3          6          0          0    0.00751    0.00151
                People         61        254      0.898     0.0695      0.208      0.091
                   Bus          5          7      0.595      0.429      0.558      0.403
                   Car         12         46      0.412       0.13      

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  2,  5,  6, 11])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7feb935e18e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
   

In [ ]:
import os, csv, datetime
from ultralytics import YOLO

DATA_YAML = '/content/drive/MyDrive/yolo_project/datasets/COCO-Tiny/data.yaml'

# best.pt path for each experiment
weights_list = {
    "baseline_exdark"    : "/content/drive/MyDrive/yolo_project/outputs/exdark_experiment 1.0/weights/best.pt",
    "clahe_exdark"       : "/content/drive/MyDrive/yolo_project/outputs/exdark_experiment_CLAHE/weights/best.pt",
    "gamma_exdark"       : "/content/drive/MyDrive/yolo_project/outputs/exdark_gamma_experimen 1.0/weights/best.pt",
    "clahe_gamma_exdark" : "/content/drive/MyDrive/yolo_project/outputs/exdark_enhanced_experiment/weights/best.pt",
    "finetune_bdd_night" : "/content/drive/MyDrive/yolo_project/outputs/bdd100k_night_finetune2/weights/best.pt",
}

IMGSZ  = 640
BATCH  = 16
DEVICE = 0
WORKERS = 0

# Save results to CSV
os.makedirs("/content/drive/MyDrive/yolo_project/outputs", exist_ok=True)
csv_path = f"/content/drive/MyDrive/yolo_project/outputs/generalization_results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

rows = []
print("\n=== Start batch validation on:", DATA_YAML, "===\n")
for name, w in weights_list.items():
    try:
        print(f"▶ Evaluating [{name}] : {w}")
        model = YOLO(w)
        res = model.val(
            data=DATA_YAML,
            imgsz=IMGSZ,
            batch=BATCH,
            device=DEVICE,
            workers=WORKERS,
            verbose=False,
        )
        row = {
            "model": name,
            "weights": w,
            "precision": float(res.box.mp),        # mean precision
            "recall": float(res.box.mr),           # mean recall
            "mAP50": float(res.box.map50),         # mAP@0.5
            "mAP50-95": float(res.box.map),        # mAP@0.5:0.95
            "images": int(res.speed['images']) if 'images' in res.speed else "",
        }
        rows.append(row)
        print(f"{name} done | mAP@0.5={row['mAP50']:.3f} | mAP@0.5:0.95={row['mAP50-95']:.3f}\n")
    except Exception as e:
        print(f"{name} failed:", e, "\n")

# Writing CSV
fieldnames = ["model", "weights", "precision", "recall", "mAP50", "mAP50-95", "images"]
with open(csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for r in rows:
        writer.writerow(r)

# Sort by mAP50 and print an easy-to-read comparison table
rows_sorted = sorted(rows, key=lambda x: x["mAP50"], reverse=True)
print("=== Summary (sorted by mAP@0.5) ===")
print(f"{'Model':24s}  {'P':>6s}  {'R':>6s}  {'mAP50':>8s}  {'mAP50-95':>9s}")
for r in rows_sorted:
    print(f"{r['model']:24s}  {r['precision']:.3f}  {r['recall']:.3f}    {r['mAP50']:.3f}     {r['mAP50-95']:.3f}")

print(f"\n📄 CSV saved to: {csv_path}")


=== Start batch validation on: /content/drive/MyDrive/yolo_project/datasets/COCO-Tiny/data.yaml ===

▶ Evaluating [baseline_exdark] : /content/drive/MyDrive/yolo_project/outputs/exdark_experiment 1.0/weights/best.pt
Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 5.0±11.0 MB/s, size: 61.9 KB)
val: Scanning /content/drive/MyDrive/yolo_project/datasets/COCO-Tiny/labels/val... 128 images, 57 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 15.0it/s 8.6s
val: New cache created: /content/drive/MyDrive/yolo_project/datasets/COCO-Tiny/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 2.5it/s 3.2s
                   all        128        318      0.736      0.336      0.398      0.288
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 2.9ms postproc